In [20]:
pip install --upgrade scikit-learn numpy


  Using cached numpy-2.2.5-cp311-cp311-macosx_14_0_arm64.whl.metadata (62 kB)
Using cached numpy-2.2.5-cp311-cp311-macosx_14_0_arm64.whl (5.4 MB)
  Attempting uninstall: numpy
    Found existing installation: numpy 1.26.0
    Uninstalling numpy-1.26.0:
      Successfully uninstalled numpy-1.26.0
Note: you may need to restart the kernel to use updated packages.


In [ ]:
import pandas as pd
import numpy as np

In [139]:
files = [
    {'year': 22, 'visit_file': 'h239g.csv', 'people_file': 'h243.csv'},
    {'year': '21', 'visit_file': 'h229g.csv', 'people_file': 'h233.csv'},
    {'year': '20', 'visit_file': 'h220g.csv', 'people_file': 'h224.csv'},
    {'year': '19', 'visit_file': 'h213g.csv', 'people_file': 'h216.csv'},
    ]
dfs = []

In [140]:
def load_data(visit_file, people_file, year, dfs):
    rename_map = {
        f'OBTC{year}X': 'total_charge',  # Total charge
        f'OBSF{year}X': 'out_of_pocket',  # Out-of-pocket
        f'OBMR{year}X': 'medicare',  # Medicare
        f'OBMD{year}X': 'medicade',  # Medicaid
        f'OBPV{year}X': 'priv_insurance',  # Private insurance
        f'OBVA{year}X': 'VA_CHAMPVA',  # VA/CHAMPVA
        f'OBTR{year}X': 'tricare',  # Tricare
        f'OBOF{year}X': 'other_federal', # Other federal
        f'OBSL{year}X': 'state_local',  # State/local
        f'OBWC{year}X': 'workers_comp',  # Worker’s comp
        f'OBOT{year}X': 'other_insurance',  # Other insurance
        f'PERWT{year}F': 'person_weight',  # Person weight
        f'INSURC{year}': 'insurance'
    }

    insurance_col = f'INSURC{year}'

    df_visits = pd.read_csv(visit_file)
    df_people = pd.read_csv(people_file)
    df_people = df_people[['DUPERSID', insurance_col]]
    df_initial = df_visits.merge(df_people, on='DUPERSID', how='left')
    df_curr = df_initial.copy(deep=True)
    df_curr.rename(columns=rename_map, inplace=True)
    dfs.append(df_curr)

In [143]:
for file in files:
    load_data(file['visit_file'], file['people_file'], file['year'], dfs)

df = pd.concat(dfs, ignore_index=True)
print(df)
df.to_csv("df_all_years.csv", index=False)

            DUID  PID    DUPERSID           EVNTIDX  EVENTRN FFEEIDX  PANEL  \
0        2460002  101  2460002101  2460002101008301        8      -1     24   
1        2460002  101  2460002101  2460002101009001        9      -1     24   
2        2460006  101  2460006101  2460006101009401        7      -1     24   
3        2460006  101  2460006101  2460006101010501        8      -1     24   
4        2460006  101  2460006101  2460006101010601        8      -1     24   
...          ...  ...         ...               ...      ...     ...    ...   
1007501  2469688  102  2469688102  2469688102000201        1      -1     24   
1007502  2469689  101  2469689101  2469689101000101        1      -1     24   
1007503  2469689  101  2469689101  2469689101000301        2      -1     24   
1007504  2469689  101  2469689101  2469689101000401        2      -1     24   
1007505  2469689  101  2469689101  2469689101000501        2      -1     24   

         MPCELIG  MPCDATA  OBDATEYR  ...  insurance

In [144]:
df_full = pd.read_csv("df_all_years.csv")

/var/folders/_t/mfgk4xxn1hb8w2n3w5x65snw0000gp/T/ipykernel_62072/1144292760.py:1: DtypeWarning: Columns (5) have mixed types. Specify dtype option on import or set low_memory=False.
  df_full = pd.read_csv("df_all_years.csv")


In [158]:
df_working = df_full.copy(deep=True)

#Replaces lab test code 95 (no test received) with code 2 (no)
df_working['LABTEST_M18'] = df['LABTEST_M18'].replace(95, 2.0)
df_working['SONOGRAM_M18'] = df['SONOGRAM_M18'].replace(95, 2.0)
df_working['XRAYS_M18'] = df['XRAYS_M18'].replace(95, 2.0)
df_working['MAMMOG_M18'] = df['MAMMOG_M18'].replace(95, 2.0)
df_working['MRI_M18'] = df['MRI_M18'].replace(95, 2.0)
df_working['EKG_M18'] = df['EKG_M18'].replace(95, 2.0)
df_working['RCVVAC_M18'] = df['RCVVAC_M18'].replace(95, 2.0)

#df['LABTEST_M18'].value_counts()

In [159]:
MEPS_NA = [-1, -7, -8, -15]
df_working.replace(MEPS_NA, np.nan, inplace=True)

insurance_map = {
    1.: '<65 Any Private',
    2.: '<65 Public Only',
    3.: '<65 Uninsured',
    4.: '65+ Medicare Only',
    5.: '65+ Medicare + Private',
    6.: '65+ Medicare + Other Public',
    7.: '65+ Uninsured',
    8.: '65+ No Medicare but Public/Private'
}
df_working['insurance']
df_working['insurance'] = df_working['insurance'].map(insurance_map)

print(df_working['insurance'])

0          65+ Medicare + Other Public
1          65+ Medicare + Other Public
2                      <65 Public Only
3                      <65 Public Only
4                      <65 Public Only
                      ...             
1007501                            NaN
1007502                            NaN
1007503                            NaN
1007504                            NaN
1007505                            NaN
Name: insurance, Length: 1007506, dtype: object


In [163]:
cost_columns = [
    'total_charge',   # Total charge
    'out_of_pocket',   # Out-of-pocket (family)
    'medicare',   # Medicare
    'medicaid',   # Medicaid
    'priv_insurance',   # Private insurance
    'VA_CHAMPVA',   # VA/CHAMPVA
    'tricare',   # Tricare
    'other_federal',   # Other federal
    'state_local',   # State/local gov
    'workers_comp',   # Worker’s comp
    'other_insurance',   # Other insurance
]

df_working['COST_COVERED'] = df_working['total_charge'] - df_working['out_of_pocket']

print(df_working['COST_COVERED'])

0          248.00
1          338.00
2          130.66
3          719.61
4          205.03
            ...  
1007501    140.00
1007502    132.00
1007503    349.83
1007504    349.83
1007505    218.82
Name: COST_COVERED, Length: 1007506, dtype: float64


In [165]:
df_working.to_csv("df_working.csv", index=False)

In [ ]:
df_working = pd.read_csv("df_working.csv")

Summary Statistics

In [164]:
def weighted_mean(x, weight):
    return (x * weight).sum() / weight.sum()


summary_weighted = df_working.groupby('insurance').apply(
    lambda g: pd.Series({
        'Mean Total Cost': weighted_mean(g['total_charge'], g['person_weight']),
        'Mean Out-of-Pocket': weighted_mean(g['out_of_pocket'], g['person_weight']),
        'Mean Paid By Insurer': weighted_mean(g['COST_COVERED'], g['person_weight'])
    })
)

print(summary_weighted)

                                    Mean Total Cost  Mean Out-of-Pocket  \
insurance                                                                 
65+ Medicare + Other Public              819.462777           15.823821   
65+ Medicare + Private                   806.554945           41.200506   
65+ Medicare Only                        807.451266           40.131619   
65+ No Medicare but Public/Private       590.888647           53.383702   
65+ Uninsured                           1080.804815           95.219158   
<65 Any Private                          570.265034           70.406642   
<65 Public Only                          678.891624           13.627058   
<65 Uninsured                            569.674821          125.763535   

                                    Mean Paid By Insurer  
insurance                                                 
65+ Medicare + Other Public                   803.638955  
65+ Medicare + Private                        765.354438  
65+ Medicare 

In [35]:
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LinearRegression, Ridge, Lasso
from sklearn.ensemble import RandomForestRegressor
from sklearn.pipeline import Pipeline
from sklearn.compose import ColumnTransformer
from sklearn.preprocessing import OneHotEncoder
from sklearn.metrics import mean_squared_error, r2_score
import pandas as pd
import numpy as np

In [36]:
feature_cols = [
    'VISITTYPE', 'VSTCTGRY', 'DOCATLOC', 'DRSPLTY_M18',
    'MEDPTYPE_M18', 'SEEDOC_M18', 'SURGPROC', 'MEDPRESC', 'TELEHEALTHFLAG',
    'LABTEST_M18', 'XRAYS_M18', 'MRI_M18', 'EKG_M18', 'OBDATEYR', 'OBDATEMM', 'INSURANCE'
]

df[feature_cols] = df[feature_cols].fillna('Missing')
df[feature_cols] = df[feature_cols].astype(str)
df_model = df[feature_cols + ['OBTC22X', 'OBSF22X', 'COST_COVERED']]

X = df_model[feature_cols]
print(X)

targets = {
    'Total Cost': df_model['OBTC22X'],
    'Out-of-Pocket': df_model['OBSF22X'],
    'Cost Covered': df_model['COST_COVERED']
}

categorical_cols = X.select_dtypes(include='object').columns.tolist() + [
    'VISITTYPE', 'VSTCTGRY', 'DOCATLOC', 'DRSPLTY_M18',
    'MEDPTYPE_M18', 'SEEDOC_M18', 'SURGPROC', 'MEDPRESC', 'TELEHEALTHFLAG',
    'LABTEST_M18', 'XRAYS_M18', 'MRI_M18', 'EKG_M18', 'OBDATEYR', 'OBDATEMM'
]

preprocessor = ColumnTransformer(
    transformers=[
        ('cat', OneHotEncoder(handle_unknown='ignore'), categorical_cols)
    ],
    remainder='passthrough'
)

       VISITTYPE VSTCTGRY DOCATLOC DRSPLTY_M18 MEDPTYPE_M18 SEEDOC_M18  \
0        Missing      1.0  Missing         8.0      Missing        1.0   
1        Missing      2.0      2.0     Missing          8.0        2.0   
2        Missing      1.0  Missing        14.0      Missing        1.0   
3        Missing      2.0  Missing        14.0      Missing        1.0   
4        Missing      2.0  Missing        14.0      Missing        1.0   
...          ...      ...      ...         ...          ...        ...   
151258   Missing      1.0  Missing        14.0      Missing        1.0   
151259   Missing      5.0      2.0     Missing         15.0        2.0   
151260   Missing      1.0      2.0     Missing          2.0        2.0   
151261   Missing      2.0  Missing         6.0      Missing        1.0   
151262   Missing      2.0  Missing         6.0      Missing        1.0   

       SURGPROC MEDPRESC TELEHEALTHFLAG LABTEST_M18 XRAYS_M18 MRI_M18 EKG_M18  \
0           2.0      1.0      

In [ ]:
for target_name, y in targets.items():
    print(f"\n Prediction: {target_name}")

    # Combine features and actual target column (y is a Series with the correct column)
    df_model = pd.concat([X, y], axis=1)
    #df_with_na = df_model[df_model.isna().any(axis=1)]
    #print(df_with_na)
    #num_na_rows = df_model.isna().any(axis=1).sum()
    #print(f"{num_na_rows} rows contain at least one NA value.")
    df_model = df_model.dropna()
    print(df_model)



    X_clean = df_model[feature_cols]
    y_clean = df_model[y.name]  # Get actual column name from Series

    X_train, X_test, y_train, y_test = train_test_split(
        X_clean, y_clean, test_size=0.2, random_state=42
    )

    pipe = Pipeline(steps=[
        ('preprocessor', preprocessor),
        ('model', LinearRegression())
    ])

    pipe.fit(X_train, y_train)
    y_pred = pipe.predict(X_test)

    rmse = np.sqrt(mean_squared_error(y_test, y_pred))
    r2 = r2_score(y_test, y_pred)

    print(f"{'Linear Regression':20} | RMSE: {rmse:8.2f} | R²: {r2:.4f}")



 Prediction: Total Cost
Empty DataFrame
Columns: [VISITTYPE, VSTCTGRY, DOCATLOC, DRSPLTY_M18, MEDPTYPE_M18, SEEDOC_M18, SURGPROC, MEDPRESC, TELEHEALTHFLAG, LABTEST_M18, XRAYS_M18, MRI_M18, EKG_M18, OBDATEYR, OBDATEMM, INSURANCE, OBTC22X]
Index: []
0 rows contain at least one NA value.
       VISITTYPE VSTCTGRY DOCATLOC DRSPLTY_M18 MEDPTYPE_M18 SEEDOC_M18  \
0        Missing      1.0  Missing         8.0      Missing        1.0   
1        Missing      2.0      2.0     Missing          8.0        2.0   
2        Missing      1.0  Missing        14.0      Missing        1.0   
3        Missing      2.0  Missing        14.0      Missing        1.0   
4        Missing      2.0  Missing        14.0      Missing        1.0   
...          ...      ...      ...         ...          ...        ...   
151258   Missing      1.0  Missing        14.0      Missing        1.0   
151259   Missing      5.0      2.0     Missing         15.0        2.0   
151260   Missing      1.0      2.0     Missing 